In [1]:
import nltk
import numpy as np
import pandas as pd
import json

In [2]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec 
model = Word2Vec.load("word2vec.model")
wordvectors = model.wv

unable to import 'smart_open.gcs', disabling that module


In [3]:
from embeddings import *

In [4]:
getCentroidTableVectors("data/tables/re_tables-0001.json")

D:\uni\Information Retrieval\ir-table-retrieval\notebooks\embeddings.py:87: RuntimeWarning: divide by zero encountered in true_divide
  tableVectors = tableVectors / cnt
D:\uni\Information Retrieval\ir-table-retrieval\notebooks\embeddings.py:87: RuntimeWarning: invalid value encountered in true_divide
  tableVectors = tableVectors / cnt


[ inf -inf  inf  inf  inf -inf -inf  inf  inf  inf  inf  inf -inf  inf
 -inf -inf -inf  inf -inf  inf -inf -inf -inf -inf  inf  inf  inf -inf
  inf -inf  inf  inf  inf -inf  inf -inf -inf -inf -inf -inf  inf  inf
 -inf  inf  inf -inf  inf  inf  inf -inf -inf  inf  inf  inf -inf  inf
  inf  inf  inf  inf  inf -inf  inf  inf -inf -inf  inf  inf  inf -inf
 -inf -inf  inf -inf -inf -inf  inf -inf -inf  inf  inf -inf -inf  inf
 -inf -inf -inf  inf -inf -inf  inf -inf  inf -inf  inf -inf -inf  inf
  inf -inf -inf  inf  inf -inf  inf  inf  inf -inf -inf -inf -inf  inf
  inf -inf -inf  inf  inf -inf  inf -inf  inf -inf -inf  inf -inf -inf
  inf -inf -inf  inf  inf  inf -inf  inf -inf -inf -inf -inf -inf -inf
  inf -inf -inf  inf  inf  inf  inf -inf -inf -inf -inf -inf  inf  inf
  inf -inf -inf  inf -inf -inf  inf -inf  inf  inf  inf  inf  inf  inf
 -inf  inf -inf  inf  inf -inf -inf  inf -inf -inf -inf  inf -inf -inf
 -inf  inf -inf  inf  inf -inf -inf  inf -inf  inf -inf -inf -inf  inf
  inf 

In [5]:
getCentroidQueryVectors("data/queries/queries.txt")

array([[ 2.37173793e-25, -2.47455948e-25,  4.12775117e-25, ...,
         1.83808167e-25, -1.53699500e-26,  1.15549203e-25],
       [ 1.15195842e-24, -3.06617634e-25, -3.42359911e-25, ...,
         3.18768177e-25,  3.32447628e-25,  3.00286066e-25],
       [ 3.29214568e-24, -2.78950934e-24,  1.39189207e-24, ...,
         2.68437379e-24,  1.77518026e-24,  2.50445232e-24],
       ...,
       [ 2.85933018e-02, -2.95352519e-01,  3.21121380e-01, ...,
        -1.04310744e-01, -4.51904039e-03, -1.43892363e-01],
       [ 3.66322696e-01, -3.21821628e-01,  2.83711409e-01, ...,
        -1.76659403e-01, -1.78646470e-01,  4.59678491e-01],
       [-9.38199759e-02, -3.49026704e+00, -3.07172582e-01, ...,
        -1.65977776e+00, -1.44173491e+00, -1.93865886e+00]])